In [1]:
import torch
import cv2
import importipynb
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as f
import argparse
from spikingjelly.clock_driven import neuron, surrogate, functional
import torch.nn.functional as F
import os
from MIOU_BIOU import Evaluator
from torch.utils.tensorboard import SummaryWriter
import PSPnet_model 
from PIL import Image
from torch.utils.data import Dataset
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

importing Jupyter notebook from MIOU_BIOU.ipynb
importing Jupyter notebook from PSPnet_model.ipynb


In [2]:
class MyDataset(Dataset):#继承了Dataset子类
    def __init__(self,input_root,label_root,transform_img=None,transform_label=None):
        #分别读取输入/标签图片的路径信息
        self.input_root=input_root
        self.input_files=os.listdir(input_root)#列出指定路径下的所有文件
        
        self.label_root=label_root
        self.label_files=os.listdir(label_root)
 
        self.transforms_img=transform_img
        self.transforms_label=transform_label
    def __len__(self):
        #获取数据集大小
        return len(self.input_files)
    def __getitem__(self, index):
        #根据索引(id)读取对应的图片
        input_img_path=os.path.join(self.input_root,self.input_files[index])
        input_img=Image.open(input_img_path).convert('RGB')
     
        label_img_path=os.path.join(self.label_root,self.label_files[index])
        label_img=Image.open(label_img_path)
        label_img=self.transforms_label(label_img)
        
        label_img =np.array(label_img)
        label_img = torch.FloatTensor(label_img)
        input_img=self.transforms_img(input_img)

        return (input_img,label_img)

In [3]:
def train_model(epoch):         #训练模型
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    IOU = []
    B_IOU = []
    sum_loss = 0.0
    total = 0.0
    Evaluator_trian =Evaluator(2)

    for i, data in enumerate(trainloader, 0):
        # 准备数据
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs= net(inputs)

        labels = torch.squeeze(labels)
        labels =labels.long()
        outputs  = torch.sigmoid(outputs)
        loss = criterion(outputs, labels)     #计算损失函数
        loss.backward()
        optimizer.step()
        
        outputs =torch.argmax(outputs,axis=1)
        labels = labels.data.cpu().numpy()
        outputs = outputs.data.cpu().numpy()
    
        Evaluator_trian.add_batch(labels,outputs) #将一batch输入进计算类

        sum_loss += loss.item()
        print('[epoch:%d, iter:%d] Loss: %.03f | '
              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1)))
        
    iou = Evaluator_trian.Mean_Intersection_over_Union() #计算MIOU,BIOU指标
    b_iou = Evaluator_trian.boundary_iouget()
    print("训练集miou:", iou)
    print("训练集Boundary IoU:", b_iou)   
    writer.add_scalar('训练集损失', sum_loss / (i + 1),epoch+1) 
    writer.add_scalar('训练集MIOU', iou, epoch+1)
    writer.add_scalar('训练集BIOU', b_iou, epoch+1)

In [4]:
def test_mdoel(epoch):           #测试集结果
    print("Waiting Test!")
    IOU = []
    Evaluator_test=Evaluator(2)
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            #outputs,aux = net(images)
            outputs = net(images)
            outputs  = torch.sigmoid(outputs)
            outputs = outputs.data.cpu().numpy()
            labels = labels.data.cpu().numpy()
            outputs = np.argmax(outputs,axis=1)
            Evaluator_test.add_batch(labels,outputs)
              
        iou = Evaluator_test.Mean_Intersection_over_Union()       #计算MIOU,BIOU指标
        b_iou = Evaluator_test.boundary_iouget()
        print("测试集miou:", iou)
        print("测试集Boundary IoU:", b_iou)
        #print('测试IOU：',mean_iou)
        writer.add_scalar('测试集MIOU', iou, epoch+1)
        writer.add_scalar('测试集BIOU', b_iou, epoch+1)
        # 将每次测试结果实时写入acc.txt文件中


In [5]:

EPOCH = 100#遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 10  #批处理尺寸(batch_size)
LR = 0.02     #学习率

writer = SummaryWriter('./hhpath/to/log')

input_root='C:\\Users\\aizir\\PSPnet\\weizmann_horse_db\\horse_train'
label_root='C:\\Users\\aizir\\PSPnet\\weizmann_horse_db\\mask_train'

input_test = 'C:\\Users\\aizir\\PSPnet\\weizmann_horse_db\\horse_test'
label_test='C:\\Users\\aizir\\PSPnet\\weizmann_horse_db\\mask_test'
# 准备数据集并预处理

transform_img = transforms.Compose([
    transforms.Resize((160, 200), interpolation=Image.BILINEAR),
    transforms.ToTensor(),
 
])
transform_label= transforms.Compose([
    transforms.Resize((160, 200), interpolation=Image.NEAREST),

])
                  #构建Dataset
dataset_train= MyDataset(input_root, label_root, transform_img=transform_img,transform_label=transform_label )
print(len(dataset_train))
trainloader=torch.utils.data.DataLoader(dataset_train,batch_size=10, shuffle=True)
dataset_test= MyDataset(input_test , label_test, transform_img=transform_img,transform_label=transform_label)
testloader=torch.utils.data.DataLoader(dataset_test,batch_size=10, shuffle=False)

# 模型定义-pspnet
net =PSPnet_model.PSPNet(classes=2,poolchannl=2048,outpoolchannl=512).to(device)


# 定义损失函数和优化方式
weights=torch.FloatTensor([1.6,3.7]).to(device)
criterion = nn.CrossEntropyLoss(weight = weights)  #损失函数为交叉熵，多用于多分类问题
optimizer =optim.Adam(net.parameters(), weight_decay=0.000001, lr=LR)#Adam优化器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCH)

# 训练
if __name__ == "__main__":

    print("Start Training, pspnet")  # 定义遍历数据集的次数
    for epoch in range(pre_epoch, EPOCH):
        train_model(epoch)
        # 每训练完一个epoch测试一下准确率
        test_mdoel(epoch)
        scheduler.step()
        print(optimizer.state_dict()['param_groups'][0]['lr'])
    torch.save(net.state_dict(),"./pspnetepoch_100.pkl")           #保存最终模型
    print("Training Finished, TotalEPOCH=%d" % EPOCH)
                    
net.eval()


C:\Users\aizir\anaconda3\envs\pytorch\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
<string>:5: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


278
Start Training, pspnet

Epoch: 1


C:\Users\aizir\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


[epoch:1, iter:1] Loss: 0.675 | 
[epoch:1, iter:2] Loss: 0.699 | 
[epoch:1, iter:3] Loss: 0.686 | 
[epoch:1, iter:4] Loss: 0.672 | 
[epoch:1, iter:5] Loss: 0.649 | 
[epoch:1, iter:6] Loss: 0.633 | 
[epoch:1, iter:7] Loss: 0.624 | 
[epoch:1, iter:8] Loss: 0.613 | 
[epoch:1, iter:9] Loss: 0.615 | 
[epoch:1, iter:10] Loss: 0.614 | 
[epoch:1, iter:11] Loss: 0.608 | 
[epoch:1, iter:12] Loss: 0.607 | 
[epoch:1, iter:13] Loss: 0.599 | 
[epoch:1, iter:14] Loss: 0.596 | 
[epoch:1, iter:15] Loss: 0.594 | 
[epoch:1, iter:16] Loss: 0.590 | 
[epoch:1, iter:17] Loss: 0.589 | 
[epoch:1, iter:18] Loss: 0.590 | 
[epoch:1, iter:19] Loss: 0.586 | 
[epoch:1, iter:20] Loss: 0.582 | 
[epoch:1, iter:21] Loss: 0.581 | 
[epoch:1, iter:22] Loss: 0.582 | 
[epoch:1, iter:23] Loss: 0.578 | 
[epoch:1, iter:24] Loss: 0.575 | 
[epoch:1, iter:25] Loss: 0.571 | 
[epoch:1, iter:26] Loss: 0.569 | 
[epoch:1, iter:27] Loss: 0.567 | 
[epoch:1, iter:28] Loss: 0.564 | 
confusion matrix:
 [[5136565. 1563520.]
 [ 693865. 150205

[epoch:7, iter:183] Loss: 0.416 | 
[epoch:7, iter:184] Loss: 0.417 | 
[epoch:7, iter:185] Loss: 0.418 | 
[epoch:7, iter:186] Loss: 0.418 | 
[epoch:7, iter:187] Loss: 0.419 | 
[epoch:7, iter:188] Loss: 0.419 | 
[epoch:7, iter:189] Loss: 0.420 | 
[epoch:7, iter:190] Loss: 0.419 | 
[epoch:7, iter:191] Loss: 0.418 | 
[epoch:7, iter:192] Loss: 0.418 | 
[epoch:7, iter:193] Loss: 0.418 | 
[epoch:7, iter:194] Loss: 0.417 | 
[epoch:7, iter:195] Loss: 0.417 | 
[epoch:7, iter:196] Loss: 0.417 | 
confusion matrix:
 [[5945922.  754163.]
 [ 225257. 1970658.]]
sum 8896000.0
训练集miou: 0.763288246048191
训练集Boundary IoU: 0.3048298767383364
Waiting Test!
confusion matrix:
 [[995694. 136387.]
 [ 56328. 379591.]]
sum 1568000.0
测试集miou: 0.7505518102257605
测试集Boundary IoU: 0.2960832317238174
0.019759167619387476

Epoch: 8
[epoch:8, iter:197] Loss: 0.410 | 
[epoch:8, iter:198] Loss: 0.406 | 
[epoch:8, iter:199] Loss: 0.401 | 
[epoch:8, iter:200] Loss: 0.401 | 
[epoch:8, iter:201] Loss: 0.401 | 
[epoch:8, iter:

...

[epoch:80, iter:2232] Loss: 0.329 | 
[epoch:80, iter:2233] Loss: 0.329 | 
[epoch:80, iter:2234] Loss: 0.329 | 
[epoch:80, iter:2235] Loss: 0.329 | 
[epoch:80, iter:2236] Loss: 0.329 | 
[epoch:80, iter:2237] Loss: 0.329 | 
[epoch:80, iter:2238] Loss: 0.329 | 
[epoch:80, iter:2239] Loss: 0.329 | 
[epoch:80, iter:2240] Loss: 0.329 | 
confusion matrix:
 [[6578493.  121592.]
 [  32146. 2163769.]]
sum 8896000.0
训练集miou: 0.9554131107457228
训练集Boundary IoU: 0.7824887283278101
Waiting Test!
confusion matrix:
 [[1090153.   41928.]
 [  25724.  410195.]]
sum 1568000.0
测试集miou: 0.899996023580655
测试集Boundary IoU: 0.6173753472672474
0.0019098300562505278

Epoch: 81
[epoch:81, iter:2241] Loss: 0.330 | 
[epoch:81, iter:2242] Loss: 0.329 | 
[epoch:81, iter:2243] Loss: 0.330 | 
[epoch:81, iter:2244] Loss: 0.330 | 
[epoch:81, iter:2245] Loss: 0.330 | 
[epoch:81, iter:2246] Loss: 0.329 | 
[epoch:81, iter:2247] Loss: 0.329 | 
[epoch:81, iter:2248] Loss: 0.329 | 
[epoch:81, iter:2249] Loss: 0.329 | 
[epoch:8

[epoch:86, iter:2401] Loss: 0.327 | 
[epoch:86, iter:2402] Loss: 0.327 | 
[epoch:86, iter:2403] Loss: 0.327 | 
[epoch:86, iter:2404] Loss: 0.327 | 
[epoch:86, iter:2405] Loss: 0.327 | 
[epoch:86, iter:2406] Loss: 0.327 | 
[epoch:86, iter:2407] Loss: 0.327 | 
[epoch:86, iter:2408] Loss: 0.327 | 
confusion matrix:
 [[6592758.  107327.]
 [  28321. 2167594.]]
sum 8896000.0
训练集miou: 0.9604725642468237
训练集Boundary IoU: 0.8033503987681684
Waiting Test!
confusion matrix:
 [[1089482.   42599.]
 [  25857.  410062.]]
sum 1568000.0
测试集miou: 0.8989113793180777
测试集Boundary IoU: 0.6183825917505683
0.000951729475339807

Epoch: 87
[epoch:87, iter:2409] Loss: 0.327 | 
[epoch:87, iter:2410] Loss: 0.327 | 
[epoch:87, iter:2411] Loss: 0.327 | 
[epoch:87, iter:2412] Loss: 0.328 | 
[epoch:87, iter:2413] Loss: 0.328 | 
[epoch:87, iter:2414] Loss: 0.328 | 
[epoch:87, iter:2415] Loss: 0.328 | 
[epoch:87, iter:2416] Loss: 0.328 | 
[epoch:87, iter:2417] Loss: 0.327 | 
[epoch:87, iter:2418] Loss: 0.327 | 
[epoch:8

[epoch:92, iter:2570] Loss: 0.326 | 
[epoch:92, iter:2571] Loss: 0.326 | 
[epoch:92, iter:2572] Loss: 0.326 | 
[epoch:92, iter:2573] Loss: 0.326 | 
[epoch:92, iter:2574] Loss: 0.326 | 
[epoch:92, iter:2575] Loss: 0.326 | 
[epoch:92, iter:2576] Loss: 0.326 | 
confusion matrix:
 [[6603122.   96963.]
 [  25469. 2170446.]]
sum 8896000.0
训练集miou: 0.9641996747711065
训练集Boundary IoU: 0.8195188687074719
Waiting Test!
confusion matrix:
 [[1091104.   40977.]
 [  28989.  406930.]]
sum 1568000.0
测试集miou: 0.8965144173569716
测试集Boundary IoU: 0.6136472812056973
0.00031416838871369055

Epoch: 93
[epoch:93, iter:2577] Loss: 0.327 | 
[epoch:93, iter:2578] Loss: 0.326 | 
[epoch:93, iter:2579] Loss: 0.326 | 
[epoch:93, iter:2580] Loss: 0.326 | 
[epoch:93, iter:2581] Loss: 0.326 | 
[epoch:93, iter:2582] Loss: 0.326 | 
[epoch:93, iter:2583] Loss: 0.326 | 
[epoch:93, iter:2584] Loss: 0.326 | 
[epoch:93, iter:2585] Loss: 0.326 | 
[epoch:93, iter:2586] Loss: 0.326 | 
[epoch:93, iter:2587] Loss: 0.326 | 
[epoch

[epoch:98, iter:2739] Loss: 0.325 | 
[epoch:98, iter:2740] Loss: 0.325 | 
[epoch:98, iter:2741] Loss: 0.325 | 
[epoch:98, iter:2742] Loss: 0.325 | 
[epoch:98, iter:2743] Loss: 0.325 | 
[epoch:98, iter:2744] Loss: 0.325 | 
confusion matrix:
 [[6607437.   92648.]
 [  23484. 2172431.]]
sum 8896000.0
训练集miou: 0.9659915596361077
训练集Boundary IoU: 0.8284037909770895
Waiting Test!
confusion matrix:
 [[1092807.   39274.]
 [  30171.  405748.]]
sum 1568000.0
测试集miou: 0.8970544990425577
测试集Boundary IoU: 0.6154508588159769
1.9732715717284423e-05

Epoch: 99
[epoch:99, iter:2745] Loss: 0.325 | 
[epoch:99, iter:2746] Loss: 0.325 | 
[epoch:99, iter:2747] Loss: 0.325 | 
[epoch:99, iter:2748] Loss: 0.325 | 
[epoch:99, iter:2749] Loss: 0.325 | 
[epoch:99, iter:2750] Loss: 0.325 | 
[epoch:99, iter:2751] Loss: 0.325 | 
[epoch:99, iter:2752] Loss: 0.325 | 
[epoch:99, iter:2753] Loss: 0.326 | 
[epoch:99, iter:2754] Loss: 0.325 | 
[epoch:99, iter:2755] Loss: 0.325 | 
[epoch:99, iter:2756] Loss: 0.325 | 
[epoch

PSPNet(
  (pool1): PyramidPool(
    (pooldown): Sequential(
      (0): AdaptiveAvgPool2d(output_size=1)
      (1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.95, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
  )
  (pool2): PyramidPool(
    (pooldown): Sequential(
      (0): AdaptiveAvgPool2d(output_size=2)
      (1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.95, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
  )
  (pool3): PyramidPool(
    (pooldown): Sequential(
      (0): AdaptiveAvgPool2d(output_size=3)
      (1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.95, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
  )
  (pool6): PyramidPool(
    (pooldown): Sequential(
      (0): AdaptiveAvgPoo

In [28]:
net.eval()                     #结果可视化
vis_path='C://Users//aizir//PSPnet//weizmann_horse_db//horse_test/horse309.png'
mask_path='C:\\Users\\aizir\\PSPnet\\weizmann_horse_db\\mask_test\\horse309.png'
vis=Image.open(vis_path).convert('RGB')
vis_img=transform_img(vis)
vis_img= vis_img.unsqueeze(dim=0).cuda()
img = np.array([[0 for col in range(200)] for row in range(160)])

dd=Image.open(mask_path)                      #绘制mask图像
dd=transform_label(dd)

dd =np.array(dd)    
dd= torch.FloatTensor(dd)
print(dd.shape)
#print(dd.shape)
for i in range(160):
    for j in range(200):
        if dd[i][j]==1:
            img[i][j] = 255
        else:
            img[i][j] = 0
            
imageD = Image.fromarray(img)
imageD.show() 
imageD = imageD.convert('L').save("mask4.png")

pre =net(vis_img)
pre =torch.squeeze(pre)
pre  = torch.sigmoid(pre).data.cpu().numpy()   #绘制预测输出图像
img = np.array([[0 for col in range(200)] for row in range(160)])
for i in range(160):
    for j in range(200):
        if pre[0][i][j]<pre[1][i][j]:
            img[i][j] =255
        else:
            img[i][j] =0
imageX = Image.fromarray(img)
imageX.show() 
imageX = imageX.convert('L').save("res4.png")


torch.Size([160, 200])


C:\Users\aizir\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [ ]:
mask_path='C:\\Users\\aizir\\PSPnet\\weizmann_horse_db\\mask_test\\horse315.png'
dd=Image.open(mask_path)                      #绘制mask图像

dd =np.array(dd)  

In [7]:
!tensorboard  --logdir=./hhpath/to/log --port 8120

^C


##### 